# ARIMAX

In [15]:
import pickle
TRAIN=12
# Run for the one or the other
# read dict where train size is TRAIN
input_file = open('dict_of_chunk_series_with_test_and_train_' + str(TRAIN) + '.pickle', 'rb')
dict_of_chunk_series_with_test_and_train = pickle.load(input_file)
input_file.close()

# read dict where train size is expanding
#second_input_file = open('dict_of_chunk_series_with_expanding_test_and_steady_train.pickle', 'rb')
#dict_of_chunk_series_with_test_and_train = pickle.load(second_input_file)
#second_input_file.close()


In [25]:
import pandas as pd
import pmdarima as pm
import numpy as np
import copy

# Expand the previously created dictionary (dict_of_chunk_series_with_test_and_train) to also hold the prediction series next to the train and the test series (and threshold values for test)

dict_of_chunk_series_with_test_and_train_and_forecast = copy.deepcopy(dict_of_chunk_series_with_test_and_train)
dict_of_chunk_series_with_forecast_df = {}
accuracy_dict_for_chunk_iterations = {}


for i, chunk in enumerate(dict_of_chunk_series_with_test_and_train_and_forecast):
    dict_of_chunk_series_with_forecast_df[chunk] = {}
    accuracy_dict_for_chunk_iterations[chunk] = {}
    
    for i, chunk_iteration in enumerate(dict_of_chunk_series_with_test_and_train_and_forecast[chunk]):
        
        TEST = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TEST_LIST_MEDIAN"].size
        tp, tn, fp, fn = 0, 0, 0, 0

        ####################
        # ARIMAX for High Alarms
        ####################
      

        current_train_list_high = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TRAIN_LIST_MAX"] 
        current_test_list_high = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TEST_LIST_MAX"]
        current_train_list_exog_high = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TRAIN_LIST_MEDIAN"].values.reshape(-1, 1)
        current_test_list_exog_high = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TEST_LIST_MEDIAN"].values.reshape(-1, 1)
        arimax_high = pm.auto_arima(current_train_list_high, X=current_train_list_exog_high)
        forecast_arimax_high = pd.Series(arimax_high.predict(TEST, X=current_test_list_exog_high), index=[*range(i+TRAIN,i+TRAIN+TEST,1)], name="forecast_list_arimax_high")
        dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["FORECAST_LIST_ARIMAX_HIGH"] = forecast_arimax_high

        ####################
        # ARIMAX for Low Alarms
        ####################
        
        current_train_list_low = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TRAIN_LIST_MIN"] 
        current_test_list_low = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TEST_LIST_MIN"] 
        current_train_list_exog_low = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TRAIN_LIST_MEDIAN"].values.reshape(-1, 1)
        current_test_list_exog_low = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TEST_LIST_MEDIAN"].values.reshape(-1, 1)
        arimax_low = pm.auto_arima(current_train_list_low, X=current_train_list_exog_low)
        forecast_arimax_low = pd.Series(arimax_low.predict(TEST, X=current_test_list_exog_low), index=[*range(i+TRAIN,i+TRAIN+TEST,1)], name="forecast_list_arimax_low")
        dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["FORECAST_LIST_ARIMAX_LOW"] = forecast_arimax_low


        # extract threshold series 
        threshold_high_for_test_list = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["THRESHOLD_HIGH_FOR_TEST_LIST"]
        threshold_low_for_test_list = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["THRESHOLD_LOW_FOR_TEST_LIST"]
        
        # write to dict_of_chunk_series_with_forecast_df dataframe
        all_dict_lists_as_df = pd.concat([current_test_list_high,forecast_arimax_high,threshold_high_for_test_list,current_test_list_low,forecast_arimax_low,threshold_low_for_test_list],axis=1)
        dict_of_chunk_series_with_forecast_df[chunk][chunk_iteration] = all_dict_lists_as_df

        ####################
        # Add information whether alarm was triggered
        ####################

        df_for_chunk_iteration = dict_of_chunk_series_with_forecast_df[chunk][chunk_iteration]
        
        # True alarms
        df_for_chunk_iteration['high_alarm_triggered'] = np.where(df_for_chunk_iteration['test_list_max'] > df_for_chunk_iteration['threshold_high_for_test_list'] ,1,0)
        df_for_chunk_iteration['low_alarm_triggered'] = np.where(df_for_chunk_iteration['test_list_min'] < df_for_chunk_iteration['threshold_low_for_test_list'] ,1,0)
        
        
        # ARIMAX forecast
        df_for_chunk_iteration['high_alarm_triggered_forecast_arimax'] = np.where(df_for_chunk_iteration['forecast_list_arimax_high'] > df_for_chunk_iteration['threshold_high_for_test_list'] ,1,0)
        df_for_chunk_iteration['low_alarm_triggered_forecast_arimax'] = np.where(df_for_chunk_iteration['forecast_list_arimax_low'] < df_for_chunk_iteration['threshold_low_for_test_list'] ,1,0)
        #write to dict_of_chunk_series_with_forecast_and_alarm_df dataframe
        dict_of_chunk_series_with_forecast_df[chunk][chunk_iteration] = df_for_chunk_iteration

        #Still working on the following part
        ####################
        # Calculate Confusion Matrix
        ####################

        # true high alarms triggered already selected above
        # select predicted high alarms
        column_index_of_high_alarm_triggered_forecast_arimax = df_for_chunk_iteration.columns.get_loc("high_alarm_triggered_forecast_arimax")
        
        # create df with bot as column - only needed rows (test only) # What does this comment mean?
        # overwrite ARIMA low_alarms with ARIMAX forecast
        high_alarms = df_for_chunk_iteration.iloc[TRAIN:,[column_index_of_high_alarm_triggered,column_index_of_high_alarm_triggered_forecast_arimax]]
        
        for row_in_high_alarms_arimax in high_alarms.iterrows():

            if row_in_high_alarms[1][0] and row_in_high_alarms[1][1]:
                tp +=1
                # print("tp", tp)
            if row_in_high_alarms[1][0] and not row_in_high_alarms[1][1]:
                fn +=1
                # print("fn", fn)
            if not row_in_high_alarms[1][0] and row_in_high_alarms[1][1]:
                fp +=1
                # print("fp", fp)
            if not row_in_high_alarms[1][0] and not row_in_high_alarms[1][1]:
                tn +=1
                # print("tn",tn)
        
        a_new_row = {"TP":tp,"FN":fn,"FP":fp,"TN":tn}
        a_new_row_series = pd.Series(a_new_row,name="accuracy_high_alarms_arimax")

        accurracy_matrix_df_for_chunk_iteration = accurracy_matrix_df_for_chunk_iteration.append(a_new_row_series)


In [28]:
dict_of_chunk_series_with_forecast_and_alarm_df[chunk][12]

,test_list_max,forecast_list_arimax_high,threshold_high_for_test_list,test_list_min,forecast_list_arimax_low,threshold_low_for_test_list,high_alarm_triggered,low_alarm_triggered,high_alarm_triggered_forecast_arimax,low_alarm_triggered_forecast_arimax
13,86.0,82.479776,120.0,71.0,70.488104,60.0,0,0,0,0


In [34]:

accuracy_dict_for_chunk_iterations = {}

for i,chunk in enumerate(dict_of_chunk_series_with_forecast_and_alarms):
    accuracy_dict_for_chunk_iterations[chunk] = {}
       
    for i,chunk_iteration in enumerate(dict_of_chunk_series_with_forecast_and_alarms[chunk]):
        
        # select column of dataframe but only where relevant (starting after train size)
        df_for_chunk_iteration = dict_of_chunk_series_with_forecast_and_alarms[chunk][chunk_iteration]

        #####################
        # ARIMA
        #####################
        
        tp, tn, fp, fn = 0, 0, 0, 0
        accurracy_matrix_df_for_chunk_iteration = pd.DataFrame(columns=["TP","FN","FP","TN"])

        #####################
        # ARIMA: High alarms
        #####################
        
        # select true high alarms triggered
        column_index_of_high_alarm_triggered = df_for_chunk_iteration.columns.get_loc("high_alarm_triggered")

        # select predicted high alarms
        column_index_of_high_alarm_triggered_forecast = df_for_chunk_iteration.columns.get_loc("high_alarm_triggered_forecast")
        
        # create df with bot as column - only needed rows (test only)
        high_alarms = df_for_chunk_iteration.iloc[TRAIN:,[column_index_of_high_alarm_triggered,column_index_of_high_alarm_triggered_forecast]]
        
        for row_in_high_alarms in high_alarms.iterrows():

            if row_in_high_alarms[1][0] and row_in_high_alarms[1][1]:
                tp +=1
                # print("tp", tp)
            if row_in_high_alarms[1][0] and not row_in_high_alarms[1][1]:
                fn +=1
                # print("fn", fn)
            if not row_in_high_alarms[1][0] and row_in_high_alarms[1][1]:
                fp +=1
                # print("fp", fp)
            if not row_in_high_alarms[1][0] and not row_in_high_alarms[1][1]:
                tn +=1
                # print("tn",tn)
        
        a_new_row = {"TP":tp,"FN":fn,"FP":fp,"TN":tn}
        a_new_row_series = pd.Series(a_new_row,name="accuracy_high_alarms")

        accurracy_matrix_df_for_chunk_iteration = accurracy_matrix_df_for_chunk_iteration.append(a_new_row_series)

        #####################
        # ARIMA: Low alarms
        #####################
        # Reset tp, tn, fp, fn
        tp, tn, fp, fn = 0, 0, 0, 0
        # select true low alarms triggered
        column_index_of_low_alarm_triggered = df_for_chunk_iteration.columns.get_loc("low_alarm_triggered")

        # select predicted low alarms
        column_index_of_low_alarm_triggered_forecast = df_for_chunk_iteration.columns.get_loc("low_alarm_triggered_forecast")
        
        # create df with bot as column - only needed rows (test only) # What does this comment mean?
        low_alarms = df_for_chunk_iteration.iloc[TRAIN:,[column_index_of_low_alarm_triggered,column_index_of_low_alarm_triggered_forecast]]
        
        for row_in_low_alarms in low_alarms.iterrows():

            if row_in_low_alarms[1][0] and row_in_low_alarms[1][1]:
                tp +=1
                # print("tp", tp)
            if row_in_low_alarms[1][0] and not row_in_low_alarms[1][1]:
                fn +=1
                # print("fn", fn)
            if not row_in_low_alarms[1][0] and row_in_low_alarms[1][1]:
                fp +=1
                # print("fp", fp)
            if not row_in_low_alarms[1][0] and not row_in_low_alarms[1][1]:
                tn +=1
                # print("tn",tn)
        
        a_new_row = {"TP":tp,"FN":fn,"FP":fp,"TN":tn}
        a_new_row_series = pd.Series(a_new_row,name="accuracy_low_alarms")
        
        accurracy_matrix_df_for_chunk_iteration = accurracy_matrix_df_for_chunk_iteration.append(a_new_row_series)

        #####################
        # ARIMAX
        #####################
        
        # Reset tp, tn, fp, fn 
        tp, tn, fp, fn = 0, 0, 0, 0

        #####################
        # ARIMAX: High alarms
        #####################
        
        # true high alarms triggered already selected above
        # select predicted high alarms
        column_index_of_high_alarm_triggered_forecast_arimax = df_for_chunk_iteration.columns.get_loc("high_alarm_triggered_forecast_arimax")
        
        # create df with bot as column - only needed rows (test only) # What does this comment mean?
        # overwrite ARIMA low_alarms with ARIMAX forecast
        high_alarms = df_for_chunk_iteration.iloc[TRAIN:,[column_index_of_high_alarm_triggered,column_index_of_high_alarm_triggered_forecast_arimax]]
        
        for row_in_high_alarms_arimax in high_alarms.iterrows():

            if row_in_high_alarms[1][0] and row_in_high_alarms[1][1]:
                tp +=1
                # print("tp", tp)
            if row_in_high_alarms[1][0] and not row_in_high_alarms[1][1]:
                fn +=1
                # print("fn", fn)
            if not row_in_high_alarms[1][0] and row_in_high_alarms[1][1]:
                fp +=1
                # print("fp", fp)
            if not row_in_high_alarms[1][0] and not row_in_high_alarms[1][1]:
                tn +=1
                # print("tn",tn)
        
        a_new_row = {"TP":tp,"FN":fn,"FP":fp,"TN":tn}
        a_new_row_series = pd.Series(a_new_row,name="accuracy_high_alarms_arimax")

        accurracy_matrix_df_for_chunk_iteration = accurracy_matrix_df_for_chunk_iteration.append(a_new_row_series)

        #####################
        # ARIMAX: Low alarms
        #####################
        # Reset tp, tn, fp, fn
        tp, tn, fp, fn = 0, 0, 0, 0

        # true low alarms triggered already selected above
        # select predicted low alarms
        column_index_of_low_alarm_triggered_forecast_arimax = df_for_chunk_iteration.columns.get_loc("low_alarm_triggered_forecast_arimax")
        
        # create df with bot as column - only needed rows (test only) # What does this comment mean?
        # overwrite ARIMA low_alarms with ARIMAX forecast
        low_alarms = df_for_chunk_iteration.iloc[TRAIN:,[column_index_of_low_alarm_triggered,column_index_of_low_alarm_triggered_forecast_arimax]]
        
        for row_in_low_alarms in low_alarms.iterrows():

            if row_in_low_alarms[1][0] and row_in_low_alarms[1][1]:
                tp +=1
                # print("tp", tp)
            if row_in_low_alarms[1][0] and not row_in_low_alarms[1][1]:
                fn +=1
                # print("fn", fn)
            if not row_in_low_alarms[1][0] and row_in_low_alarms[1][1]:
                fp +=1
                # print("fp", fp)
            if not row_in_low_alarms[1][0] and not row_in_low_alarms[1][1]:
                tn +=1
                # print("tn",tn)
        
        a_new_row = {"TP":tp,"FN":fn,"FP":fp,"TN":tn}
        a_new_row_series = pd.Series(a_new_row,name="accuracy_low_alarms_arimax")
        
        accurracy_matrix_df_for_chunk_iteration = accurracy_matrix_df_for_chunk_iteration.append(a_new_row_series)
        
        # Write accuracy matrix into dictionary
        accuracy_dict_for_chunk_iterations[chunk][chunk_iteration] = accurracy_matrix_df_for_chunk_iteration
    


In [35]:
# Example to check for all iterations of one chunk
accuracy_dict_for_chunk_iterations[chunk][11]

,TP,FN,FP,TN
accuracy_high_alarms,0,0,0,2
accuracy_low_alarms,0,0,0,2
accuracy_high_alarms_arimax,0,0,0,2
accuracy_low_alarms_arimax,0,0,0,2


In [36]:
 # Transform nested dictionary accuracy_dict_for_chunk_iterations into a data frame using pd.concat()
 # Aggregate so that TP, FN, FP, and TN are summed up, grouped by accuracy_high_alarms and accuracy_low_alarms

accuracy_dict_for_chunks = {}

for i,chunk in enumerate(dict_of_chunk_series_with_forecast_and_alarms):
    df = pd.concat(accuracy_dict_for_chunk_iterations[chunk], axis=0)
    df = df.reset_index(level=0, drop=True).reset_index(drop=False).rename(columns={'index':'ACCURACY_TYPE'})
    df = df.groupby('ACCURACY_TYPE').agg('sum')
    accuracy_dict_for_chunks[chunk] = df


In [37]:
# Example to check aggregated accurracy matrix of one chunk (containing TP, FN, FP, and TN summed up by alarm type)
accuracy_dict_for_chunks[chunk]

,TP,FN,FP,TN
ACCURACY_TYPE,,,,
accuracy_high_alarms,0,0,0,4
accuracy_high_alarms_arimax,0,0,0,4
accuracy_low_alarms,0,0,0,4
accuracy_low_alarms_arimax,0,0,0,4


## Documentation snippets

so that they are not lost. to be updated

In [ ]:
# Needed Adaption for following cell:
# Change list_of_chunk_value_series from List to Dictionary
# The CHUNK_ID is used as key and in this step one key holds three series: the vital parameter series, the low threshold series and the high threshold series. They need the same "sampling rate" - so that the high threshold with index 0 is the high threshold that applies at the time of the vital parameter with index 0 

# Vital parameter Series:
# index                       |     0 |    1  |   2   | ...
# ----------------------------------------------------- ...
# firstChunk - Vital Parameter|  95.0 |  90.5 |  91.0 | ...

# Threshold High Series:
# index                       |     0 |    1  |   2   | ...
# ----------------------------------------------------- ...
# firstChunk - Th. High       |  120.0 |  120.0 |  110.0 | ...

# Threshold Low Series:
# index                       |     0 |    1  |   2   | ...
# ----------------------------------------------------- ...
# firstChunk - Th. Low        |  70.0 |  70.0 |  60.0 | ...

In [ ]:
# Needed Adaption for following cell:
# Currently we only have a true values list and a predictions list. But we are not interested in whether the prediction is exactly the true value. We want to see if the prediction value also triggers an alarm if the true value does. Therefore we need the threshold values that apply at the time of the respective prediction/true value.
# A Final version should hold the following informations that can be traced back to a specific Chunk ID:
# * List of true values (vital parameters in test list)
# * List of Threshold High (for the time at which the predictions take place)
# * List of Threshold Low (for the time at which the predictions take place)
# * Arima Predictions (the predictions for the true values based on the train values)

# Our thoughts:
# Currently prediction looks as follows (two colums as TRAIN is 2; two rows as two chunk_value_series are created for our chunk (containing 15 values)):

#   | 0                                         | 1
# 0 | first prediction for chunk_value_series 1 | second prediction for chunk_value_series 1
# 1 | first prediction for chunk_value_series 2 | second prediction for chunk_value_series 2

# We wanted to add the last index of the train_list and the CHUNK_ID in a nested way to these predictions so that we can trace them back to the thresholds that apply at the time of the prediction

#   | CHUNK_ID | Time ref. | 0                                 | 1
# 0 |  xxxx    | 11        | 1st pred. for chunk_value_series 1| 2nd pred. for chunk_value_series 1 
# 1 |  xxxx    | 12        | 1st pred. for chunk_value_series 2| 2nd pred. for chunk_value_series 2
